In [64]:
import pandas as pd
dframe = pd.read_csv("steam_csv")
nuevo_df = dframe[['genres', 'precio', 'sentiment', "release_year"]].copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28852 non-null  object 
 1   precio        29676 non-null  float64
 2   sentiment     24953 non-null  object 
 3   release_year  29783 non-null  float64
dtypes: float64(2), object(2)
memory usage: 1004.3+ KB


In [65]:
df1 = nuevo_df.dropna(subset=['genres'])

In [66]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28852 entries, 0 to 32133
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        28852 non-null  object 
 1   precio        26710 non-null  float64
 2   sentiment     22536 non-null  object 
 3   release_year  28549 non-null  float64
dtypes: float64(2), object(2)
memory usage: 1.1+ MB


In [68]:
df1 = df1[df1['release_year']>= 2015]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20789 entries, 0 to 32133
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        20789 non-null  object 
 1   precio        19410 non-null  float64
 2   sentiment     15731 non-null  object 
 3   release_year  20789 non-null  float64
dtypes: float64(2), object(2)
memory usage: 812.1+ KB


In [69]:
df1 = df1.dropna(subset=['genres'])
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20789 entries, 0 to 32133
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        20789 non-null  object 
 1   precio        19410 non-null  float64
 2   sentiment     15731 non-null  object 
 3   release_year  20789 non-null  float64
dtypes: float64(2), object(2)
memory usage: 812.1+ KB


In [70]:
df1 = df1.dropna(subset=['precio'])
df1.head()

,genres,precio,sentiment,release_year
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",4.99,NaN,2018.0
3,"['Action', 'Adventure', 'Casual']",0.99,NaN,2017.0
5,"['Action', 'Adventure', 'Simulation']",3.99,Mixed,2018.0
6,"['Free to Play', 'Indie', 'Simulation', 'Sports']",9.99,NaN,2018.0
7,"['Free to Play', 'Indie', 'Simulation', 'Sports']",18.99,NaN,2018.0


In [71]:
df1['sentiment'].unique()

array([nan, 'Mixed', '1 user reviews', '3 user reviews',
       'Mostly Positive', '6 user reviews', '5 user reviews',
       '2 user reviews', 'Overwhelmingly Positive', 'Very Negative',
       'Very Positive', 'Mostly Negative', 'Positive', '9 user reviews',
       '8 user reviews', '7 user reviews', 'Negative', '4 user reviews',
       'Overwhelmingly Negative'], dtype=object)

In [72]:
valores_a_borrar = ['1 user reviews', '3 user reviews', '6 user reviews', '5 user reviews', '2 user reviews', '4 user reviews', '8 user reviews', '7 user reviews', '9 user reviews']

In [74]:
df1 = df1[~df1['sentiment'].isin(valores_a_borrar)]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13041 entries, 0 to 32132
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        13041 non-null  object 
 1   precio        13041 non-null  float64
 2   sentiment     8224 non-null   object 
 3   release_year  13041 non-null  float64
dtypes: float64(2), object(2)
memory usage: 509.4+ KB


In [75]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error


def prediccion(genero, sentiment):
    # Filtro el DF por género y sentimiento
    datos_filtrados = df1[df1['genres'].apply(lambda x: genero in x)]
    
    # Codificamos las categorías de sentimiento utilizando Label Encoding
    le = LabelEncoder()
    datos_filtrados['sentiment_encoded'] = le.fit_transform(datos_filtrados['sentiment'])
    
    # Divido los datos en conjuntos de entrenamiento y prueba
    X = datos_filtrados[['precio', 'release_year']]
    y = datos_filtrados['sentiment_encoded']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Verificar si hay suficientes datos para entrenar el modelo
    if len(X_train) < 2:
        return None, None
    
    # Creo y entreno el modelo de regresión lineal
    modelo = LinearRegression()
    modelo.fit(X_train, y_train)
    
    # Realizamos predicciones en el conjunto de prueba
    y_pred = modelo.predict(X_test)
    
    # Calculao el RMSE
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    # Retorna el precio y el RMSE
    return datos_filtrados['precio'].values[0], rmse

In [81]:
print(prediccion("Simulation", "Positive"))

(4.99, 3.0986038653345482)


C:\Users\emily\AppData\Local\Temp\ipykernel_19764\2331766796.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_filtrados['sentiment_encoded'] = le.fit_transform(datos_filtrados['sentiment'])
